## (주)디유푸드 리뷰 긁어오는 코드

 ### Local에 저장된 DB, 엑셀 파일에서 (주)다유푸드 컬럼에 해당하는 site_url을 열어서 리뷰 긁어오기

In [ ]:
import pandas as pd
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 엑셀 파일 경로
excel_file_path = r"C:\Users\USER\Desktop\5_6\core_pj\data\환자식_imageO_siteO_108개.xlsx"

# 엑셀 파일 로드
df_excel = pd.read_excel(excel_file_path)

# '디유푸(주)'의 'site_url' 값을 필터링
urls = df_excel[df_excel['제조사명'] == '디유푸드(주)']['site_url'].tolist()

# 드라이버 설정
driver = wb.Chrome(service=Service(ChromeDriverManager().install()))

reviews_data = []

for url in urls:
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    # 광고 페이지 닫기
    ad_close_selector = '#is2popup1 > div.is2_popup-content > div > div.is2_popup-buttons > button.is2_popup-button.is2_popup-button-today'
    try:
        ad_close_button = driver.find_element(By.CSS_SELECTOR, ad_close_selector)
        ad_close_button.click()
        time.sleep(1)  # 광고 닫기 대기
    except Exception as e:
        print("광고 페이지가 발견되지 않음:", e)

    # 후기 탭 클릭
    review_tab_selector = '#saleson > section > div.item_tab.widowScrolled > div > ul > li:nth-child(3) > a > span.txt'
    review_tab = driver.find_element(By.CSS_SELECTOR, review_tab_selector)
    driver.execute_script("arguments[0].click();", review_tab)
    time.sleep(2)  # 페이지 로딩 대기

    # 상품명 가져오기
    product_name_selector = '#saleson > section > div.view_top.container > div > div.col-12.col-lg-6.view_info > div.tit_area > p.title'
    product_name = driver.find_element(By.CSS_SELECTOR, product_name_selector).text

    reviews = []
    page = 1

    while True:
        try:
            print(f"Scraping page {page} of {url}...")

            # 리뷰 텍스트 가져오기
            review_texts = driver.find_elements(By.CSS_SELECTOR, '#review-list > li > div > div > div.review_origin > div.txt_area > div')
            for review in review_texts:
                reviews.append(review.text)

            # 다음 페이지 버튼 클릭
            next_button = driver.find_element(By.CSS_SELECTOR, '#review-list > ul > li.page-item.next > a')
            if next_button.is_displayed():
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(2)  # 페이지 로딩 대기
                page += 1
            else:
                break
        except Exception as e:
            print(f"Exception occurred: {e}")
            break

    for review in reviews:
        reviews_data.append({
            "Product Name": product_name,
            "Review": review,
            "URL": url
        })

# 데이터프레임 생성
df_reviews = pd.DataFrame(reviews_data)

# 크롤링 결과 출력
print(df_reviews)

# 최종 결과를 엑셀로 저장
output_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\DayouFoodReview.xlsx'
df_reviews.to_excel(output_path, index=False)
print(f"다유푸드 리뷰 데이터가 {output_path}에 저장되었습니다.")


# 크롬 드라이버 종료
driver.quit()


## 리뷰 코드 합치기 코드

In [3]:
import pandas as pd

# 파일 경로
file1_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\hyundaiGreenFoodAllReview.xlsx'
file2_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\DayouFoodReview.xlsx'

# 엑셀 파일 로드
df1 = pd.read_excel(file1_path)
df2 = pd.read_excel(file2_path)

# 두 데이터프레임을 이어 붙이기
combined_df = pd.concat([df1, df2], ignore_index=True)

# 결합된 데이터프레임을 새로운 엑셀 파일로 저장
output_file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\possibleReview.xlsx'
combined_df.to_excel(output_file_path, index=False)

print(f"결합된 데이터가 {output_file_path}에 저장되었습니다.")


결합된 데이터가 C:\Users\USER\Desktop\5_6\core_pj\data\crawlingOrNogada\allReviewData.xlsx에 저장되었습니다.
